In [1]:
import os
import pandas as pd

In [6]:
# Ruta a tu archivo Excel
excel_file = './data/Estadisticas_historicas_comercializacion.xlsx'

os.makedirs('data', exist_ok=True)

# ajusta estos valores según tu archivo:
skip_top = 6     # filas de encabezado a omitir
skip_bottom = 3  # filas de pie de página a omitir

df_import = pd.read_excel(
    excel_file,
    sheet_name='IMPORTACION',
    engine='openpyxl',
    skiprows=skip_top,
    skipfooter=skip_bottom
)
df_consumo = pd.read_excel(
    excel_file,
    sheet_name='CONSUMO',
    engine='openpyxl',
    skiprows=skip_top,
    skipfooter=skip_bottom
)

df_import.to_csv('data/importacion.csv', index=False, encoding='utf-8-sig')
df_consumo.to_csv('data/consumo.csv',    index=False, encoding='utf-8-sig')

print("Listo: data/importacion.csv y data/consumo.csv creados sin header ni footer.")

Listo: data/importacion.csv y data/consumo.csv creados sin header ni footer.
